<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/District_and_Provience_wise_NCRF_forecast_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: add the base dir

Step 2 : Add the provience and district Shapefile


In [20]:
# Install the required libraries
!pip install rasterio pyproj pandas matplotlib imageio requests geopandas

import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta
import imageio
from IPython.display import HTML
import requests
import geopandas as gpd
from shapely.geometry import box
from matplotlib.animation import FuncAnimation

# Set root_dir to a fixed value
root_dir = "20240724"

# Create the frames directory
frames_dir = os.path.join(root_dir, "FRAMES")
os.makedirs(frames_dir, exist_ok=True)

# Define the base URI
uri_base = f"https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/{root_dir}/"

# Define the forecast origin date
forecast_origin_date = datetime.strptime(root_dir, "%Y%m%d")

# Function to pad numbers
def pad2(num):
    return str(num).zfill(2)

def pad3(num):
    return str(num).zfill(3)

# Generate the list of URIs
tiff_urls = []
for i in range(1, 42):
    if i <= 25:
        time_step_hours = i * 3
        file_suffix = 'R4KMC.tif'
    else:
        time_step_hours = i * 3 - 2
        file_suffix = 'R12.5KMC.tif'

    time_step_str = pad3(time_step_hours)
    forecast_hour_index = pad2(i)
    forecast_time = forecast_origin_date + timedelta(hours=time_step_hours)
    forecast_time_str = forecast_time.strftime('%Y%m%d%H')

    uri = f"{uri_base}NCMRWF_Nepal_{forecast_time_str}F{forecast_hour_index}O{root_dir}00H{time_step_str}{file_suffix}"
    tiff_urls.append(uri)

print("Generated TIFF URLs:", tiff_urls)

# Function to download and save TIFF files
def download_tiff_files(urls, download_dir):
    tiff_files = []
    for url in urls:
        filename = os.path.join(download_dir, os.path.basename(url))
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            tiff_files.append(filename)
        else:
            print(f"Failed to download {url}")
    return tiff_files

# Download the TIFF files
tiff_files = download_tiff_files(tiff_urls, root_dir)
print("Downloaded TIFF files:", tiff_files)

# Load the shapefiles
prov_shapefile_path = '/content/hermes_NPL_new_wgs_1.shp'
dist_shapefile_path = '/content/hermes_NPL_new_wgs_2.shp'

prov_gdf = gpd.read_file(prov_shapefile_path)
dist_gdf = gpd.read_file(dist_shapefile_path)

# Ensure the shapefiles are in the same coordinate reference system (CRS) as your data
prov_gdf = prov_gdf.to_crs(epsg=4326)
dist_gdf = dist_gdf.to_crs(epsg=4326)

# Function to plot precipitation data from TIFF files and save to PNG frames
def animate_precipitation_data_tiff_no_background(tiff_files, shapefile_gdf, shapefile_name, output_dir):
    for i, file_path in enumerate(tiff_files):
        with rasterio.open(file_path) as src:
            precipitation = src.read(1)
            bounds = src.bounds

            fig, ax = plt.subplots(figsize=(12, 8))
            im = ax.imshow(precipitation, cmap='viridis', extent=[bounds.left, bounds.right, bounds.bottom, bounds.top], vmin=0, vmax=100)
            ax.axis('off')  # Turn off the axis

            # Add title and colorbar
            filename = os.path.basename(file_path)
            parts = filename.split('_')
            timestamp = parts[2].split('F')[0]  # Extract timestamp
            forecastIndex = parts[2].split('F')[1].split('O')[0]  # Extract forecast index
            forecastHrsPart = parts[2].split('H')[1]  # Extract forecast hours part
            forecastHrs = forecastHrsPart.split('R')[0]  # Extract forecast hours
            title = timestamp + 'UTC-F' + forecastIndex + 'H' + forecastHrs + '(+5:45 Nepal)'

            # Add a background to the title
            plt.title(title, fontsize=18, pad=20, backgroundcolor='white')

            norm = mcolors.Normalize(vmin=0, vmax=100)
            cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.05, fraction=0.025, norm=norm)
            cbar.set_label('Precipitation (mm)', fontsize=14)
            cbar.ax.yaxis.set_tick_params(labelsize=12)  # Adjust font size of color bar values

            # Plot the shapefile on top of the precipitation data
            ax.set_ylim(bounds.top, bounds.bottom)
            shapefile_gdf.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=1)

            # Save each frame with transparent background
            frame_filename = os.path.join(output_dir, f'frame_{i:04d}.png')
            plt.savefig(frame_filename, bbox_inches='tight', dpi=400, transparent=True)
            plt.close(fig)

    print("Frames saved successfully.")

    # Create an animated GIF from the frames with a delay between frames
    frame_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.png')])
    gif_filename = os.path.join(output_dir, f'precipitation_animation_{shapefile_name}.gif')
    with imageio.get_writer(gif_filename, mode='I', duration=1) as writer:  # 1-second delay
        for frame_file in frame_files:
            image = imageio.v2.imread(frame_file)
            writer.append_data(image)

    print(f"Animation saved as {gif_filename}.")

    # Function to update each frame
    def update(frame):
        im.set_data(plt.imread(frame_files[frame]))
        return im,

    # Create a figure without the outer frame
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.axis('off')  # Turn off the axis

    # Display the first frame
    im = ax.imshow(plt.imread(frame_files[0]))

    # Create the animation with the specified interval
    ani = FuncAnimation(fig, update, frames=len(frame_files), interval=1000)

    # Display the animation
    return HTML(ani.to_html5_video())

# Call the function to plot precipitation data for provinces
prov_frames_dir = os.path.join(root_dir, "provinces_frames")
os.makedirs(prov_frames_dir, exist_ok=True)
animate_precipitation_data_tiff_no_background(tiff_files, prov_gdf, 'provinces', prov_frames_dir)

# Call the function to plot precipitation data for districts
dist_frames_dir = os.path.join(root_dir, "districts_frames")
os.makedirs(dist_frames_dir, exist_ok=True)
animate_precipitation_data_tiff_no_background(tiff_files, dist_gdf, 'districts', dist_frames_dir)


Generated TIFF URLs: ['https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072403F01O2024072400H003R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072406F02O2024072400H006R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072409F03O2024072400H009R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072412F04O2024072400H012R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072415F05O2024072400H015R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072418F06O2024072400H018R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072421F07O2024072400H021R4KMC.tif', 'https://storage.googleapis.com/ncmrwf-nepal/Forecast/NCMRWF/20240724/NCMRWF_Nepal_2024072500F08O2024072400H024R4KMC.

KeyboardInterrupt: 

Add the shapefiles of district and provience


In [ ]:
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image
import os

# Load the shapefiles for province and district boundaries
def load_boundaries():
    province_shapefile_path = '/content/hermes_NPL_new_wgs_1.shp'
    district_shapefile_path = '/content/hermes_NPL_new_wgs_2.shp'

    # Load shapefiles into GeoDataFrames
    gdf_province = gpd.read_file(province_shapefile_path).to_crs(epsg=4326)
    gdf_district = gpd.read_file(district_shapefile_path).to_crs(epsg=4326)

    return gdf_province, gdf_district

# Function to merge frames for a given duration with boundaries
def merge_frames_with_boundaries(tiff_files, duration_hours, output_dir, gdf_province, gdf_district, boundary_type):
    # Create a directory for merged frames
    merged_frames_dir = os.path.join(output_dir, "merged_frames")
    os.makedirs(merged_frames_dir, exist_ok=True)

    # Create a list to store images for merging
    images = []

    print(f"Processing {len(tiff_files)} TIFF files for {boundary_type} boundaries.")

    for i, file_path in enumerate(tiff_files):
        try:
            print(f"Processing file {i+1}/{len(tiff_files)}: {file_path}")

            with rasterio.open(file_path) as src:
                precipitation = src.read(1)
                bounds = src.bounds

                # Convert the precipitation data to an image
                fig, ax = plt.subplots(figsize=(12, 8))
                im = ax.imshow(precipitation, cmap='viridis', extent=[bounds.left, bounds.right, bounds.bottom, bounds.top], vmin=0, vmax=100)
                ax.axis('off')  # Turn off the axis

                # Add title and colorbar
                filename = os.path.basename(file_path)
                parts = filename.split('_')
                timestamp = parts[2].split('F')[0]  # Extract timestamp
                forecastIndex = parts[2].split('F')[1].split('O')[0]  # Extract forecast index
                forecastHrsPart = parts[2].split('H')[1]  # Extract forecast hours part
                forecastHrs = forecastHrsPart.split('R')[0]  # Extract forecast hours
                title = timestamp + 'UTC-F' + forecastIndex + 'H' + forecastHrs
                plt.title(title, fontsize=18, pad=20, backgroundcolor='white')

                norm = mcolors.Normalize(vmin=0, vmax=100)
                cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.05, fraction=0.025, norm=norm)
                cbar.set_label('Precipitation (mm)', fontsize=14)
                cbar.ax.yaxis.set_tick_params(labelsize=12)  # Adjust font size of color bar values

                # Plot boundaries
                if boundary_type == "provinces":
                    gdf_province.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=1)
                elif boundary_type == "districts":
                    gdf_district.plot(ax=ax, edgecolor='blue', facecolor='none', linewidth=1)

                # Flip y-axis if necessary
                ax.set_ylim(ax.get_ylim()[::-1])

                # Save the frame as a PNG image
                frame_filename = os.path.join(merged_frames_dir, f'frame_{i:04d}.png')
                plt.savefig(frame_filename, bbox_inches='tight', dpi=400, transparent=True)
                plt.close(fig)

                # Append the image to the list
                images.append(Image.open(frame_filename))

                print(f"Saved frame {i+1}/{len(tiff_files)}.")

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

    # Create a merged image with a specific grid layout
    if images:
        # Determine the number of rows and columns needed
        num_images = len(images)
        cols = 2
        rows = (num_images + cols - 1) // cols

        # Get dimensions of the first image
        img_width, img_height = images[0].size

        # Create a new blank image with enough space for the grid
        merged_image = Image.new('RGBA', (cols * img_width, rows * img_height))

        for index, img in enumerate(images):
            row = index // cols
            col = index % cols
            x = col * img_width
            y = row * img_height
            merged_image.paste(img, (x, y))

        merged_image_path = os.path.join(output_dir, f'merged_precipitation_{boundary_type}_{duration_hours}_hours.png')
        merged_image.save(merged_image_path)
        print(f"Merged image with {boundary_type} boundaries saved as {merged_image_path}")

# Load boundaries
gdf_province, gdf_district = load_boundaries()

# Directories for different forecast durations
durations = [24, 72, 120]  # In hours for 1 day, 3 days, and 5 days
for duration in durations:
    # Update these directories as per your setup
    duration_frames_dir = os.path.join(root_dir, f"{duration}_hours")

    # Merge frames for provinces
    prov_duration_frames_dir = os.path.join(duration_frames_dir, "provinces")
    os.makedirs(prov_duration_frames_dir, exist_ok=True)
    merge_frames_with_boundaries(filter_tiff_files_by_duration(tiff_files, duration), duration, prov_duration_frames_dir, gdf_province, gdf_district, "provinces")

    # Merge frames for districts
    dist_duration_frames_dir = os.path.join(duration_frames_dir, "districts")
    os.makedirs(dist_duration_frames_dir, exist_ok=True)
    merge_frames_with_boundaries(filter_tiff_files_by_duration(tiff_files, duration), duration, dist_duration_frames_dir, gdf_province, gdf_district, "districts")
